In [22]:
import json

In [23]:
zeroShot = json.load(open('gsm_results_llama_0_shot_train.json'))
zeroShotCoT = json.load(open('gsm_results_llama_0_shot_cot_train.json'))
print(len(zeroShot), len(zeroShotCoT))

7473 7473


In [24]:
# get only those were cot is correct
corrCoT = {k: v for k, v in enumerate(zeroShotCoT) if v['label'] == True}
print(len(corrCoT))
zero = {k: zeroShot[k] for k in corrCoT if zeroShot[k]['label'] == False}
print(len(zero))
corrCoT = {k: zeroShotCoT[k] for k in corrCoT if k in zero}
print(len(corrCoT))

3154
2171
2171


In [25]:
zero_keys = list(zero.keys())
cot_keys = list(corrCoT.keys())
zero_keys[120], cot_keys[120]

(393, 393)

In [26]:
zero_keys = list(zero.keys())
cot_keys = list(corrCoT.keys())
dataset_1 = {}
for k, _ in enumerate(zero_keys):
    zk = zero_keys[k]
    ck = cot_keys[k]
    dataset_1[k] = [
        {"prompt": corrCoT[ck]['prompt'].split("Let's think step-by-step")[0].strip(),
         "response": "Let's think step-by-step: " + ".".join(corrCoT[ck]['response'].split(".")[1:]).strip(),
         "example_id": k,
        },
        {"prompt": zero[zk]['prompt'],
         "response": zero[zk]['response'],
         "example_id": k,
        }
    ]
    


In [14]:
# with open('data.json', 'w') as d:
#     json.dump(dataset, d, indent=4)

In [27]:
# get only those were cot is correct
corrCoT = {k: v for k, v in enumerate(zeroShotCoT) if v['label'] == True}
print(len(corrCoT))
zero = {k: zeroShot[k] for k in corrCoT if zeroShot[k]['label'] == True}
print(len(zero))
corrCoT = {k: zeroShotCoT[k] for k in corrCoT if k in zero}
print(len(corrCoT))

3154
983
983


In [28]:
zero_keys = list(zero.keys())
cot_keys = list(corrCoT.keys())
dataset_2 = {}
for k, _ in enumerate(zero_keys):
    zk = zero_keys[k]
    ck = cot_keys[k]
    dataset_2[k] = [
        {"prompt": zero[zk]['prompt'],
         "response": zero[zk]['response'],
         "example_id": k,
        },
        {"prompt": corrCoT[ck]['prompt'].split("Let's think step-by-step")[0].strip(),
         "response": "Let's think step-by-step: " + ".".join(corrCoT[ck]['response'].split(".")[1:]).strip(),
         "example_id": k,
        }
    ]

In [29]:
dataset = {**dataset_1, **dataset_2}

In [30]:
with open('data.json', 'w') as d:
    json.dump(dataset, d, indent=4)

In [4]:
import torch
def typo_loss(logprobs: torch.FloatTensor) -> torch.FloatTensor:
    """
    Args:
        logprobs: Shape (n_constitutions, n_responses)

    Returns:
        loss: Average cross-entropy loss.
    """
    logsumexp_row = torch.logsumexp(logprobs, dim=1, keepdim=True) # across responses
    logsumexp_col = torch.logsumexp(logprobs, dim=0, keepdim=True) # across constitutions
   
    logits_row = logprobs - logsumexp_row
    logits_col = logprobs - logsumexp_col

    labels_row = torch.arange(logits_row.shape[0], dtype=torch.long).to(logprobs.device) 
    labels_col = torch.arange(logits_col.shape[0], dtype=torch.long).to(logprobs.device)
   
    loss_row = torch.nn.functional.cross_entropy(logits_row[0, :], labels_row[0], reduction="mean")
    # loss_col = F.cross_entropy(logits_col.t(), labels_col, reduction="mean") # transpose col
    
    return loss_row

In [8]:
logprobs = torch.tensor([[-.1, -3],[-1000, -.1]])
typo_loss(logprobs)

tensor(0.0536)